In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.transform import resize

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, AveragePooling2D,Activation, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils.multiclass import unique_labels

from sklearn.model_selection import KFold, StratifiedKFold

import rasterio
from rasterio.plot import reshape_as_image

#seed set to allow for reproducibility within the results
seed = 2505
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


General Methods

In [2]:
#simple CSV loader
def load_samples(csv_file):
    data = pd.read_csv(os.path.join(csv_file))
    data = data[['FileName','Label','ClassName']]
    file_names = list(data.iloc[:,0])
    #Get labels withing second column
    labels = list(data.iloc[:,1])
    samples=[]
    for samp,lab in zip(file_names,labels):
        samples.append([samp,lab])
    return samples

#helper function to report  a images details
def image_details(image):
  num_bands = img_dataset.count
  print('Number of bands in image: {n}\n'.format(n=num_bands))

  # How many rows and columns?
  rows, cols = img_dataset.shape
  print('Image size is: {r} rows x {c} columns\n'.format(r=rows, c=cols))

  # What driver was used to open the raster?
  driver = img_dataset.driver
  print('Raster driver: {d}\n'.format(d=driver))

  # What is the raster's projection?
  proj = img_dataset.crs
  print('Image projection:')
  print(proj)

def plot_confusion_matrix(y_true, y_pred, classes, class_dict,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues,model_name='test'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    # convert class_id to class_name using the class_dict
    cover_names = []
    for cover_class in classes:
        cover_names.append(class_dict[cover_class])
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        pass
    #print(cm)

    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=cover_names, yticklabels=cover_names,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    #save a copy of the classification matrix
    fig.savefig('classification_matrix_' + model_name + '.png')
    return ax

def save_metrics(gridsearch_df,test_datagen,test_df,model_name,model,width,height,layers,class_count,test_count,batch,epoch):
    #predictions against test dataset
    predictions = model.predict_generator(generator=test_datagen, 
                            steps= test_count // batch,
                             verbose=1)

    eval_generator = generator(samples=test_df,batch_size=1,width=width,height=height,layers=layers,class_count=class_count)
    #determining labels
    labels = np.empty(predictions.shape)
    count = 0
    while count < len(labels):
        image_b, label_b = next(eval_generator)
        labels[count] = label_b
        count += 1

    label_index = np.argmax(labels, axis=1)     
    pred_index = np.argmax(predictions, axis=1)
    #calcualte metrics
    accuracy = accuracy_score(label_index, pred_index)
    precision = precision_score(label_index, pred_index,average='macro')
    recall = recall_score(label_index, pred_index, average='macro')
    f1 = f1_score(label_index, pred_index,average='macro')
    kappa = cohen_kappa_score(label_index, pred_index)
    gridsearch_df=gridsearch_df.append({'Modelname':model_name,'Epoch':epoch,'Batch_size':batch,'Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1_Score':f1,'Cohens_kappa':kappa},ignore_index=True)
    #create a classification report and save a copy
    clsf_report = pd.DataFrame(classification_report(y_true = label_index, y_pred = pred_index, output_dict=True)).transpose()
    clsf_report.to_csv('ClassificationReport_'+ model_name + '.csv', index= True)
    
    print('Accuracy: %f' % accuracy)
    print('Precision: %f' % precision)
    print('Recall: %f' % recall)
    print('F1 score: %f' % f1)
    print('Cohens kappa: %f' % kappa)

    # Plot non-normalized confusion matrix
    plot_confusion_matrix(label_index, pred_index, classes=np.array(list(class_names)),
                              class_dict=class_names,model_name=model_name)
    
    return gridsearch_df

Preprocessing

In [3]:
#ensure size of tile is uniform
def preprocessing(tile,label,class_count,layers,width,height):
    #print(tile.shape,"before")
    #to avoid artifacts, no antialiasing whn rescaling
    tile = resize(tile, (layers,width,height),anti_aliasing=False)
    #print(tile.shape,"resize")
    #Returns the source array reshaped into the order expected by image processing and visualization software (matplotlib, scikit-image, etc) by swapping the axes order from (bands, rows, columns) to (rows, columns, bands)
    tile = reshape_as_image(tile)
    #print(tile.shape,"reshape")
    
    #normalising tile
    tile = tile/255
    
    label = to_categorical(label,class_count)
    return tile,label

Generator

In [4]:
#Keras compatible data generator - works with rasterio compatible files (tiff only as written)
def generator(samples,batch_size,width,height,layers,class_count):
    """
    Yields next training batch, checks shape of tile, ensure image format - includes DEM
    """
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates
        #shuffle(samples)
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]

            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []

            # For each example
            for batch_sample in batch_samples:
                # Load image (X) and label (y)
                img_name = batch_sample[0]
                label = batch_sample[1]
                #load in file
                #print(os.path.join(data_path,img_name))
                with rasterio.open(os.path.join(img_name)) as ds:
                    tile=ds.read()
                #perform any preprocessing required
                tile,label = preprocessing(tile,label,class_count,layers,width,height)     

                # Add example to arrays
                X_train.append(tile)
                y_train.append(label)

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            #X_train = np.asarray(X_train).astype('float32')
            y_train = np.array(y_train)

            # The generator-y part: yield the next training batch            
            yield X_train, y_train

Create Model

In [5]:
def Basic_CNN(input_dropout_rate,hidden_dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25

    model.add(Flatten())
    model.add(Dense(192))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(hidden_dropout_rate)) #0.25

    model.add(Dense(class_count))
    model.add(Activation('softmax'))

    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    #print(model.summary())
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [6]:
def Basic_CNN_experiment(input_dropout_rate,hidden_dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))#relu
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25
    
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(hidden_dropout_rate)) #0.25

    model.add(Dense(class_count))
    model.add(Activation('softmax'))

    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    #print(model.summary())
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [7]:
def Basic_CNN_bottleneck1(input_dropout_rate,hidden_dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))#relu
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25
    
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('LeakyReLU'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(input_dropout_rate)) #0.25

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(hidden_dropout_rate)) #0.25

    model.add(Dense(class_count))
    model.add(Activation('softmax'))

    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    #print(model.summary())
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [8]:
def ResNet50(dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(class_count, activation= 'softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    print(model.summary())
    print(model.metrics_names)
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [9]:
def VGG16(dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    base_model = applications.vgg16.VGG16(weights= None, include_top=False, input_shape= input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(class_count, activation= 'softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    #print(model.summary())
    #print(model.metrics_names)
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [10]:
def Xception(dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    base_model = applications.Xception(weights=None, include_top=False, input_shape= input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(class_count, activation= 'softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    print(model.summary())
    print(model.metrics_names)
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [11]:
def EfficientNet(dropout_rate,input_shape,optimizer):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    base_model = applications.efficientnet.EfficientNetB7(include_top=False,weights=None, input_shape= input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(class_count, activation= 'softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    #print(model.summary())
    #print(model.metrics_names)
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

In [12]:
def Gullydetector_CNN(input_shape,class_count,units,activation,dropout,lr):
    #drop1 = dropout_rate, drop2 = dropout_rate_in
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation(activation = activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout)) #0.25

    model.add(Flatten())
    model.add(Dense(units = units))
    model.add(BatchNormalization())
    model.add(Activation(activation = activation))
    model.add(Dropout(dropout)) #0.25

    model.add(Dense(class_count))
    model.add(Activation(activation='softmax'))

    #model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)
    #opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=lr),
                  metrics=['accuracy'])
    #print(model.summary())
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

Model Config

In [13]:
#dictionary necessary to identify classes for confusion matrix
#class_names = dict((
#(0,  'noprospect'),
#(1, 'prospect'),
#))

class_names = dict((
(0,  'donga'),
(1, 'dried_mudflat'),
(2, 'grass_vegetation'),
(3, 'sandstone'),
(4, 'shrub_vegetation'),
))

#Model config
width = 83 #32#83  --16
height= 83 #32#83  --16
layers = 7 #7 sat #7 drone
batch_size = 25 #25
epochs = 100 #50
model_name =  'LULC_gullydetector'
verbose_setting = 1
count = 1
CV_split = 10

#hyperparameter settings
input_shape = (width, height, layers)
units = 448
activation = "relu"
dropout = 0.6
lr = 1e-05
class_count = 5
#declare all needed dataframes
gridsearch_df = pd.DataFrame(columns=['Modelname','Epoch','Batch_size','Accuracy','Precision','Recall','F1_Score','Cohens_kappa'])
data_path = './Dataset' #BinFullDroneDataset32
data_dir_list = os.listdir(data_path)
print ('the data list is: ',data_dir_list)
fulldataset_df = load_samples('Full_drone_dataset.csv')

the data list is:  ['donga', 'dried_mudflat', 'shrub_vegetation', 'sandstone', 'grass_vegetation']


Model Run

In [14]:
#train_df,test_df = train_test_split(fulldataset_df, test_size=0.2) #0.2
#test_df,valid_df = train_test_split(test_df, test_size=0.5)
#use 10% of dataset for test dataset, 10% for validation and 80% for training
fulldataset_df,test_df = train_test_split(fulldataset_df, test_size=0.1) #0.2
#test_df,valid_df = train_test_split(test_df, test_size=0.5)

Y = np.array(fulldataset_df)
X = Y[:, 0]
Y= Y[:, 1]
skf = StratifiedKFold(n_splits = CV_split) #DONT shuffle: https://stackoverflow.com/questions/69100728/how-is-it-that-the-accuracy-score-for-10-fold-cross-validation-is-worst-than-for 

#optimizer = Ftrl(lr=learn_rates)
#optimizer = SGD(lr=learn_rate,momentum=momentum)
#optimizer = Adam(learning_rate=learn_rate, epsilon=1e-05) #1.0 or 0.1

callbacks = [
    EarlyStopping(patience=10, restore_best_weights = True, monitor='val_loss', verbose=verbose_setting),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=verbose_setting),
    ModelCheckpoint('model-inprogress-multi.h5', verbose=verbose_setting, save_best_only=True, save_weights_only=True)
]

for train_index, val_index in skf.split(X,Y):

    fulldataset_df = np.array(fulldataset_df)
    train_df = fulldataset_df[train_index]
    valid_df = fulldataset_df[val_index]
    
    #counts necessary
    valid_count = len(valid_df)
    train_count = len(train_df)
    test_count = len(test_df)
    
    print('valid_count='+str(valid_count))
    print('train_count='+str(train_count))
    print('test_count='+str(test_count))
    
    smodel_name =  model_name +"_" + str(count)
    count += 1
    input_shape = (width, height, layers)
    
    #essential to create a fresh model every time
    #Create the generators 
    train_datagen = generator(samples=train_df,batch_size=batch_size,width=width,height=height,layers=layers,class_count=class_count)
    test_datagen = generator(samples=test_df,batch_size=batch_size,width=width,height=height,layers=layers,class_count=class_count)
    valid_datagen = generator(samples=valid_df,batch_size=batch_size,width=width,height=height,layers=layers,class_count=class_count)

    #create and laod model
    #model = EfficientNet(dropout_rate,input_shape,optimizer)
    model = Gullydetector_CNN(
        input_shape =input_shape,units=units, activation=activation, dropout=dropout, lr=lr,class_count = class_count
    )
    #model = Basic_CNN(dropout_rate,dropout_rate2,input_shape,optimizer)
    #model = Basic_CNN_bottleneck1(dropout_rate,dropout_rate2,input_shape,optimizer)

    #create_model_options(input_dropout_rate,hidden_dropout_rate,input_shape,optimizer)
    history = model.fit(train_datagen,steps_per_epoch=train_count // batch_size, verbose=verbose_setting, epochs=epochs,validation_data=valid_datagen,validation_steps=valid_count // batch_size,callbacks=callbacks)
    #steps_per_epoch: Total number of steps (batches of samples) to yield from generator before declaring one epoch finished and starting the next epoch. It should typically be equal to the number of unique samples of your dataset divided by the batch size.
    
    plt.figure(figsize=(10,10))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    #plt.plot(history.history['acc']) #anything but sequential models
    #plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy (%)')
    plt.xlabel('Epoch')
    plt.legend(['Train_Accuracy', 'Valid_Accuracy'], loc='upper left')
    plt.savefig('accuracy_' + smodel_name + '.png', dpi=300)

    plt.figure(figsize=(10,10))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train_Loss', 'Valid_Loss'], loc='upper left')
    plt.savefig('loss_' + smodel_name + '.png',dpi=300)
    
    #save model for later use
    model.save(smodel_name)
    
    #run the metrics and save it to a dataframe for later analysis
    gridsearch_df = save_metrics(gridsearch_df,test_datagen,test_df,smodel_name,model,width,height,layers,class_count,test_count,batch_size,epochs)
    
    #save results
    gridsearch_df.to_csv('Total_Results_'+ smodel_name + '.csv', index= True)



valid_count=3658
train_count=32918
test_count=4064


NameError: name 'keras' is not defined

output_filename = 'StudyModel32_4'
dir_name = './StudyModel32_4'
import shutil
shutil.make_archive(output_filename, 'zip', dir_name)


import zipfile
with zipfile.ZipFile('DroneDataset32.zip', 'r') as zip_ref:
    zip_ref.extractall('BinFullDroneDataset32')